In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append(r'C:\Users\hazc\Desktop\Concept-Factor\dependencies')
import concept_helper as cp
import DataUpdate
from ConceptEvent import ConceptEvent
from sqlalchemy import create_engine

c:\Users\hazc\anaconda3\Lib\site-packages\iFinDPy.pth


In [19]:
test= cp.concept_hot_index('互联网金融', '2019-01-02~2020-12-31')